## 生成数据的函数

In [2]:
def get_rectangle():
    import random
    width = random.random() 
    height = random.random()
    s = width * height
    return width, height, s

get_rectangle()

(0.02898062171287108, 0.18676513013189078, 0.005412569585507466)

## 定义数据集

In [3]:
import torch

class Dataset(torch.utils.data.Dataset):
    
    def __init__(self):
        pass
    
    def __len__(self):
        return 500
    
    def __getitem__(self, i):
        
        width, height, s = get_rectangle()
        
        x = torch.FloatTensor([width, height])
        y = torch.FloatTensor([s])
        
        return x, y
    
dataset = Dataset()

len(dataset), dataset[0]

(500, (tensor([0.3813, 0.1216]), tensor([0.0464])))

## Loader数据遍历工具

In [10]:
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=8,
                                     shuffle=True,
                                     drop_last=True)

len(loader),next(iter(loader))

(62,
 [tensor([[0.9506, 0.1355],
          [0.7047, 0.5434],
          [0.9479, 0.8158],
          [0.0732, 0.0904],
          [0.7432, 0.4849],
          [0.6078, 0.2221],
          [0.0312, 0.4173],
          [0.4079, 0.5259]]),
  tensor([[0.1288],
          [0.3830],
          [0.7734],
          [0.0066],
          [0.3604],
          [0.1350],
          [0.0130],
          [0.2145]])])

## 定义神经网络模型

In [11]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=2,out_features=32),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=32,out_features=32),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=32,out_features=1),
        )
        
    def forward(self,x):
            return self.fc(x)
        
model = Model()

model(torch.randn(8,2)).shape

torch.Size([8, 1])

In [19]:
def train():
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
    
    # MSE_loss差的平方求平均，适合要完全相等的模型
    loss_fun = torch.nn.MSELoss()
    
    model.train()
    
    for epoch in range(100):
        
        for i, (x,y) in enumerate(loader):
            
            out = model(x)
            
            loss = loss_fun(out,y)
            
            loss.backward()
            
            optimizer.step()
            
            optimizer.zero_grad()
            
        if epoch%20 == 0:
            print(epoch,loss.item())
            
    torch.save(model.state_dict(), 'model1.pth')
    
train()

0 6.74090042593889e-05
20 3.11159819830209e-05
40 3.498894511722028e-05
60 1.2907215932500549e-05
80 1.879504634416662e-05


In [23]:
@torch.no_grad()
def test():
    model.load_state_dict(torch.load('model1.pth'))
    model.eval()
    x, y = next(iter(loader))
    out = model(x)
    print(torch.cat([out,y],dim=1))
    
test()

tensor([[0.6756, 0.6707],
        [0.3611, 0.3595],
        [0.5066, 0.5038],
        [0.3818, 0.3875],
        [0.1678, 0.1620],
        [0.4522, 0.4537],
        [0.0065, 0.0060],
        [0.0437, 0.0362]])
